In [ ]:
!pip install transformers -q
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

from datasets import load_dataset
squad = load_dataset("squad", split="train[:5000]")
squad = squad.train_test_split(test_size=0.2)

import pandas as pd

notebook_login()

In [ ]:
import pandas as pd
# Convert the dataset to a dictionary
data_dict = squad["train"].to_dict()
# Create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(data_dict)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

questions = [q.strip() for q in df["question"]]
context = [q.strip() for q in df["context"]]
inputs = tokenizer(
        questions,
        context,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

offset_mapping = inputs.pop("offset_mapping")

start_positions = []
end_positions = []
answers = df['answers']
for i, offset in enumerate(offset_mapping):
    answer = answers[i]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label it (0, 0)
    if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

df["start_positions"] = start_positions
df["end_positions"] = end_positions

import pandas as pd
from datasets import Dataset
data = {'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'start_positions':start_positions,
        'end_positions': end_positions,
       }
df = pd.DataFrame(data)
df.to_csv('encoding_train.csv',index=False)
train = Dataset.from_pandas(df)

In [ ]:
import pandas as pd
# Convert the dataset to a dictionary
data_dict = squad["test"].to_dict()
# Create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(data_dict)

questions = [q.strip() for q in df["question"]]
context = [q.strip() for q in df["context"]]
inputs = tokenizer(
        questions,
        context,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

offset_mapping = inputs.pop("offset_mapping")

start_positions = []
end_positions = []
answers = df['answers']
for i, offset in enumerate(offset_mapping):
    answer = answers[i]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label it (0, 0)
    if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

df["start_positions"] = start_positions
df["end_positions"] = end_positions

data = {'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'start_positions':start_positions,
        'end_positions': end_positions,
       }
df = pd.DataFrame(data)
df.to_csv('encoding_test.csv',index=False)
test = Dataset.from_pandas(df)

In [ ]:
!pip install accelerate>=0.20.1
!pip install transformers[torch]


In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")
import accelerate

from transformers import DefaultDataCollator





Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data_collator = DefaultDataCollator()


In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=True,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,3.192090


TrainOutput(global_step=250, training_loss=3.718151611328125, metrics={'train_runtime': 8079.9386, 'train_samples_per_second': 0.495, 'train_steps_per_second': 0.031, 'total_flos': 391959300096000.0, 'train_loss': 3.718151611328125, 'epoch': 1.0})

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Load the model
loaded_model = AutoModelForQuestionAnswering.from_pretrained("samira1234/my_awesome_qa_model")
tokenizer = AutoTokenizer.from_pretrained("samira1234/my_awesome_qa_model")
# Choose a repository ID for your model
repo_id = "samira1234/my_awesome_qa_model"

# Save the model to the Hugging Face Hub
loaded_model.save_pretrained(repo_id)
tokenizer.save_pretrained(repo_id)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

('samira1234/my_awesome_qa_model/tokenizer_config.json',
 'samira1234/my_awesome_qa_model/special_tokens_map.json',
 'samira1234/my_awesome_qa_model/vocab.txt',
 'samira1234/my_awesome_qa_model/added_tokens.json',
 'samira1234/my_awesome_qa_model/tokenizer.json')

In [ ]:
!pip install transformers

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import ipywidgets as widgets
from IPython.display import display

# Load your saved model and tokenizer
saved_model_path = "samira1234/my_awesome_qa_model"  # Replace with the correct path to your saved model
model = AutoModelForQuestionAnswering.from_pretrained(saved_model_path)
tokenizer = AutoTokenizer.from_pretrained(saved_model_path)

# Create widgets
question_widget = widgets.Text(
    value='What is the capital of France?',
    placeholder='Type your question here...',
    description='Question:'
)

context_widget = widgets.Textarea(
    value='Paris is the capital of France.',
    placeholder='Type the context here...',
    description='Context:',
    layout=widgets.Layout(width='100%')
)

button_widget = widgets.Button(description="Get Answer")

output_widget = widgets.Output()

# Define function to get the answer
def get_answer(_):
    with output_widget:
        output_widget.clear_output()
        question = question_widget.value
        context = context_widget.value

        # Tokenize input and get model prediction
        inputs = tokenizer(question, context, return_tensors='pt')
        outputs = model(**inputs)
        answer_start = outputs['start_logits'].argmax().item()
        answer_end = outputs['end_logits'].argmax().item()

        # Convert token indices to answer text
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end + 1]))

        # Display the answer
        print(f"Question: {question}")
        print(f"Context: {context}")
        print(f"Answer: {answer}")

# Attach the function to the button click event
button_widget.on_click(get_answer)

# Display widgets
display(question_widget)
display(context_widget)
display(button_widget)
display(output_widget)


Text(value='What is the capital of France?', description='Question:', placeholder='Type your question here...'…

Textarea(value='Paris is the capital of France.', description='Context:', layout=Layout(width='100%'), placeho…

Button(description='Get Answer', style=ButtonStyle())

Output()